# Age of Empires 2 Player Model

This model is used to rank invidividual players in team games. This model will allow us to better balance teams by calculating the probability that team wins before we actually play.

## Todo:
- Load data from Google Sheet instead of local CSV
- Determine what EDA should be done
- Fix GridSearchCV to LogisticRegression import
- Explore adding a time component to factor in player improvement
- Determine how to better input data for predicting
- Build other classifiers

## Import dependencies

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
import matplotlib.pyplot as plt
import numpy as np

## Import Data

In [2]:
df = pd.read_csv("sample_data.csv")

# Designate all columns that are not `Outcome` as features and `Outcome` as target
X = df.loc[:, df.columns != 'Outcome']
y = df.Outcome

df.head(2)

,Shaq,Gray,Rushi,Marc,Peter,Pat,Sam,Ori,Vic,Ardy,Chad,Pat_Jr,Pat_Jr_Jr,Matt_M,Ben,Mikey,Evan,Medium_AI,Extra_Team,Outcome
0,1,0,-1,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1
1,1,0,-1,0,-1,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1


## Explore Data
This is where I should explore data. I haven't done any EDA since I created this dataset.

## Split data
Normally, I would split the data into a training set and validation set. The validation set is for checking the accuracy of the best tuned model that results from cross-validation. HOWEVER, we are working with a really small dataset. Rather than hold out datafor validation, we will assess the performance of the model through the out of sample cross validation results

In [3]:
#X_train,X_validate,y_train,y_validate=train_test_split(X,y,test_size=0.33,random_state=0)

## Double data
Since assigning teams is random, we want to ensure that the dataset is balanced. For example, when I record data, I generally always put myself as the home team (code as `1`). We mitigate this by not having an intercept term in our model. To be safe, we will still double the dataset by inverting all the records and concatenating to the orginal dataset.

Doubling happens after splitting. Therefore we would need to double the training and validation sets. We use helper functions for readability.

In [4]:
def invert_dataframe(original_dataframe):
    """Inverts the dataframe by simply multiplying all values by -1.

    Args:
        original_datatframe (df): The dataframe to be inverted.

    Returns:
        inverted_dataframe (df): The inverted dataframe.

    """
    inverted_dataframe = original_dataframe.multiply(-1)
    return(inverted_dataframe)

def combine_dataframe(first_dataframe, second_dataframe):
    """Combines the dataframes. Assumes that both dataframes have the same columns

    Args:
        first_datatframe (df): The first dataframe to be combined.
        second_datatframe (df): The second dataframe to be combined.

    Returns:
        combined_dataframe (df): The combined dataframe.

    """
    combined_dataframe = pd.concat([first_dataframe, second_dataframe])
    return(combined_dataframe)

def invert_and_combine(original_dataframe):
    """Inverts and combines the dataframes. Assumes that both dataframes have the same columns

    Args:
        original_dataframe (df): The dataframe to be inverted and combined with the original.

    Returns:
        new_dataframe (df): The combined dataframe.

    """
    inverted_dataframe = invert_dataframe(original_dataframe)
    new_dataframe = combine_dataframe(original_dataframe, inverted_dataframe)
    return(new_dataframe)
    

In [5]:
X = invert_and_combine(X)
y = invert_and_combine(y)

# These are commented out because we are not using a validation set
# X_train = invert_and_combine(X_train)
# X_validate = invert_and_combine(X_validate)
# y_train = invert_and_combine(y_train)
# y_validate = invert_and_combine(y_validate)

## Cross Validation
We will use 5 folds cross validation and GridSearch to determine the optimal hyper parameters for the logistic regression. The parameters we will test are maximum iterations, C (regularization),  penalty function, and solver optimization. We will assume NO intercept term.

In [19]:
max_iter=[100,110,120,130,140]
C = [0.01, 0.1, 1, 10, 100, 1000]
penalty = ['l1','l2']
solver = ['newton-cg', 'lbfgs', 'sag', 'saga', 'liblinear']
fit_intercept=[False]
param_grid = dict(max_iter=max_iter,C=C, penalty=penalty, fit_intercept=fit_intercept, solver=solver)

We will now execute the GridSearch over three folds. We will use `accuracy` to assess the performance of the hyperparameters.

In [20]:
lr = LogisticRegression()
grid = GridSearchCV(estimator=lr, param_grid=param_grid, cv = 5, n_jobs=-1, scoring='accuracy')
best_model = grid.fit(X, y)

print("Best score: {0} using {1}".format(round(best_model.best_score_,2), best_model.best_params_))

Best score: 0.82 using {'C': 100, 'fit_intercept': False, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}


## Assess performance
Normally, we would now use the tuned hyperparameters to ensure accuracy on the validation set. As a reminder, the model is trained on the training set and the scores are computed on the validation set. HOWEVER, as mentioned, we would rather use all of our limited data for model building, so we will not assess performance on a validation set.

In [21]:
# y_true, y_pred = y_validate, best_model.predict(X_validate)
# print(classification_report(y_true, y_pred))

We can also use a ROC curve to visualize performance. The more above the diagonal, the better. More info [here](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5)

In [22]:
# logit_roc_auc = roc_auc_score(y_validate, best_model.predict(X_validate))
# fpr, tpr, thresholds = roc_curve(y_validate, best_model.predict_proba(X_validate)[:,1])
# plt.figure()
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
# plt.show()

## Create final model
Since we are happy with the performance of our model on the validation set, we will re-fit it with all the data. There is no concern of overfitting because we already validated against data the model hadn't seen. Since we removed the validation set, the final model is the same as the model that resulted from the hyperparameter tuning.

In [23]:
# TODO: must be a cleaner way to import GridSearchCV into LogisticRegression
final_model = LogisticRegression(penalty=best_model.best_params_['penalty'], 
                                 C=best_model.best_params_['C'],
                                 fit_intercept=best_model.best_params_['fit_intercept'],
                                 max_iter=best_model.best_params_['max_iter'],
                                 solver=best_model.best_params_['solver'])

final_model = final_model.fit(X, y)

We will output the final coefficients to see how players are ranked and with what magnitude. We need to do some busy work to output a list of coefficients.

In [24]:
features = list(df.columns)
features.remove("Outcome")
[coef] = final_model.coef_.tolist()

rounded_coef = []
for number in coef:
    rounded_number = round(number, 2)
    rounded_coef.append(rounded_number)

x = zip(rounded_coef, features)
print(sorted(list(x)))

[(-11.59, 'Mikey'), (-4.08, 'Ben'), (-3.9, 'Chad'), (-3.26, 'Sam'), (-2.92, 'Matt_M'), (-2.59, 'Evan'), (-2.57, 'Ori'), (-2.22, 'Marc'), (-0.3, 'Medium_AI'), (0.0, 'Pat_Jr_Jr'), (1.16, 'Pat'), (1.52, 'Pat_Jr'), (1.66, 'Shaq'), (1.79, 'Peter'), (1.82, 'Ardy'), (3.54, 'Gray'), (3.94, 'Extra_Team'), (4.82, 'Vic'), (5.82, 'Rushi')]


## Predictions
Ultimately, we want to use this model to determine the probability of a game. Each value in the array corresponds to a person. For example, the first number is Shaq, the second number is Gray, etc.

Here we have modeled the probability that Marc (-1) beats Rushi (1).

In [25]:
result = final_model.predict_proba([[0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

print("The probability that Marc beats Rushi is {0}%".format(round(result[0][0]*100,2)))

The probability that Marc beats Rushi is 0.03%


Here we have modeled the probability that Shaq (-1) beats Gray (1).

In [26]:
result = final_model.predict_proba([[-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

print("The probability that Shaq beats Gray is {0}%".format(round(result[0][0]*100,2)))

The probability that Shaq beats Gray is 13.23%


Here we have modeled the probaility that Shaq (-1) and Gray (-1) beat Rushi

In [27]:
result = final_model.predict_proba([[-1,-1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1]])

print("The probability that Shaq and Gray beat Rushi is {0}%".format(round(result[0][0]*100,2)))

The probability that Shaq and Gray beat Rushi is 96.5%


Here we have modeled the probaility that Marc (-1) beats Sam (1)

In [28]:
result = final_model.predict_proba([[0,0,0,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]])

print("The probability that Marc beats Sam is {0}%".format(round(result[0][0]*100,2)))

The probability that Marc beats Sam is 73.76%


Here we have modeled the probaility that Marc (-1) and Sam (-1) beat Rushi (1)

In [29]:
result = final_model.predict_proba([[0,0,1,-1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1]])

print("The probability that Marc and Sam beat Rushi is {0}%".format(round(result[0][0]*100,2)))

The probability that Marc and Sam beat Rushi is 0.06%


Here we have modeled the probaility that Vic (-1) beats Rushi (1)

In [30]:
result = final_model.predict_proba([[0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0]])

print("The probability that Vic beats Rushi is {0}%".format(round(result[0][0]*100,2)))

The probability that Vic beats Rushi is 26.99%
